In [1]:

import boto3
import numpy as np
from astropy.io import fits
import gzip
import re
from io import BytesIO


def list_folders_in_bucket(bucket_name):
    s3 = boto3.client('s3')
    
    folders = set()
    paginator = s3.get_paginator('list_objects_v2')
    for result in paginator.paginate(Bucket=bucket_name, Delimiter='/'):
        # Collect common prefixes (which represent "folders")
        if 'CommonPrefixes' in result:
            for prefix in result['CommonPrefixes']:
                folder_name = prefix['Prefix'].split('/')[0]
                # Check if the folder matches the pattern 'e' followed by a five-digit integer
                if re.match(r'^e\d{5}$', folder_name):
                    folders.add(folder_name)
    
    return list(folders)

bucket_name = 'backplanetest'
folder_list = list_folders_in_bucket(bucket_name)
print(len(folder_list))

4271


In [2]:
sorted_folders = sorted(folder_list)

In [ ]:
subset = sorted_folders[0:6]

In [ ]:
# for the folders, make an image from first 100 frames 
import numpy as np
from astropy.io import fits
import gzip


s3 = boto3.client('s3')

def download_fits(bucket_name, key):
    response = s3.get_object(Bucket=bucket_name, Key=key)
    with gzip.open(BytesIO(response['Body'].read()), 'rb') as f:
        return fits.open(BytesIO(f.read()))

bucket_name = 'backplanetest'

num_images = 5

image_list = []

# file name format 
# s3://backplanetest/e00759/e00759-nd-b00-f0001-t00000-g_dose.fits.gz

for i in subset:
    print(key)
    key = f"{i}/{i}-nd-b00-f0001-t00000-g_dose.fits.gz"  # Assuming files are named like 'fits_image_1.fits', 'fits_image_2.fits', etc.
    image = download_fits(bucket_name, key)
    image_list.append(image[0].data)

# Stack the images
stacked_image = np.stack(image_list, axis=0)

# Save the stacked image as a FITS file
fits.writeto('stacked_image.fits', stacked_image, overwrite=True)

print("Stacked image saved as 'stacked_image.fits'")

In [9]:
# Initialize S3 client
s3 = boto3.client('s3')

subset = full_groups[0:100]['eclipse']
# Function to download FITS image from S3
def download_fits(bucket_name, key):
    try:
        response = s3.get_object(Bucket=bucket_name, Key=key)
        with gzip.open(BytesIO(response['Body'].read()), 'rb') as f:
            return fits.open(BytesIO(f.read()))    
    except Exception as e:
        if 'NoSuchKey' in str(e):
            print(f"Key '{key}' does not exist. Skipping...")
            return None
        else:
            print("something else went wrong")
            print(str(e))
            return None
# Bucket name
bucket_name = 'backplanetest'

# Initialize an array to hold the sum of pixel values
sum_image = None

# Add the pixel values of each image
for e in subset:
    i = str(e).zfill(5)
    key = f"e{i}-nd/e{i}-nd-b00-f3000-t00000-g_dose.fits.gz"
  # Assuming files are named like 'fits_image_1.fits', 'fits_image_2.fits', etc.
    image = download_fits(bucket_name, key)
    print(key)
    if image is not None:
        if sum_image is None:
            sum_image = np.zeros_like(image[0].data, dtype=np.float64)
        sum_image += image[0].data.astype(np.float64)

# Save the summed image as a FITS file
fits.writeto('summed_full_depth_100.fits', sum_image, overwrite=True)

print("Summed image saved as 'summed_image_2.fits'")

e00580-nd/e00580-nd-b00-f3000-t00000-g_dose.fits.gz
e00584-nd/e00584-nd-b00-f3000-t00000-g_dose.fits.gz
e00587-nd/e00587-nd-b00-f3000-t00000-g_dose.fits.gz
e00754-nd/e00754-nd-b00-f3000-t00000-g_dose.fits.gz
e00760-nd/e00760-nd-b00-f3000-t00000-g_dose.fits.gz
e00767-nd/e00767-nd-b00-f3000-t00000-g_dose.fits.gz
e00775-nd/e00775-nd-b00-f3000-t00000-g_dose.fits.gz
e00784-nd/e00784-nd-b00-f3000-t00000-g_dose.fits.gz
Key 'e00798-nd/e00798-nd-b00-f3000-t00000-g_dose.fits.gz' does not exist. Skipping...
e00798-nd/e00798-nd-b00-f3000-t00000-g_dose.fits.gz
e00807-nd/e00807-nd-b00-f3000-t00000-g_dose.fits.gz
e00814-nd/e00814-nd-b00-f3000-t00000-g_dose.fits.gz
e00826-nd/e00826-nd-b00-f3000-t00000-g_dose.fits.gz
e00834-nd/e00834-nd-b00-f3000-t00000-g_dose.fits.gz
e00841-nd/e00841-nd-b00-f3000-t00000-g_dose.fits.gz
e00848-nd/e00848-nd-b00-f3000-t00000-g_dose.fits.gz
e00861-nd/e00861-nd-b00-f3000-t00000-g_dose.fits.gz
e00869-nd/e00869-nd-b00-f3000-t00000-g_dose.fits.gz
e00882-nd/e00882-nd-b00-f3000-

In [4]:
import pandas as pd 
full_groups = pd.read_csv("every3_nuv.csv")

In [6]:
full_groups[0:100]

,eclipse
0,580
1,584
2,587
3,754
4,760
...,...
125,1886
126,1889
127,1896
128,1903


In [33]:
# Initialize S3 client
s3 = boto3.client('s3')

# subset = sorted_folders[1000:2001]
# Function to download FITS image from S3
def download_fits(bucket_name, key):
    try:
        response = s3.get_object(Bucket=bucket_name, Key=key)
        with gzip.open(BytesIO(response['Body'].read()), 'rb') as f:
            return fits.open(BytesIO(f.read()))    
    except Exception as e:
        if 'NoSuchKey' in str(e):
            print(f"Key '{key}' does not exist. Skipping...")
            return None
        else:
            print("something else went wrong")
            print(str(e))
            return None
# Bucket name
bucket_name = 'backplanetest'

for group in split_dfs: 
    # Initialize an array to hold the sum of pixel values
    sum_image = None
    print(group)
    # Add the pixel values of each image
    for e in group['eclipse']:
        i = str(e).zfill(5)
        key = f"e{i}-nd/e{i}-nd-b00-f3000-t00000-g_dose.fits.gz"        
        image = download_fits(bucket_name, key)
        print(key)
        if image is not None:
            if sum_image is None:
                sum_image = np.zeros_like(image[0].data, dtype=np.float64)
            sum_image += image[0].data.astype(np.float64)

    # Save the summed image as a FITS file
    fits.writeto(f'{i}_g100_image.fits', sum_image, overwrite=True)

    print(f"Summed image saved as '{i}_g100_image.fits'")

    eclipse
0       580
1       584
2       587
3       754
4       760
5       767
6       775
7       784
8       798
9       807
10      814
11      826
12      834
13      841
14      848
15      861
16      869
17      882
18      888
19      899
20      907
21      920
22      923
23      926
24      929
25      932
26      935
27      938
28      941
29      944
30      947
31      953
32      956
33      968
34      971
35      974
36      977
37      980
38      984
39     1412
40     1417
41     1421
42     1424
e00580-nd/e00580-nd-b00-f3000-t00000-g_dose.fits.gz
e00584-nd/e00584-nd-b00-f3000-t00000-g_dose.fits.gz
e00587-nd/e00587-nd-b00-f3000-t00000-g_dose.fits.gz
e00754-nd/e00754-nd-b00-f3000-t00000-g_dose.fits.gz
e00760-nd/e00760-nd-b00-f3000-t00000-g_dose.fits.gz
e00767-nd/e00767-nd-b00-f3000-t00000-g_dose.fits.gz
e00775-nd/e00775-nd-b00-f3000-t00000-g_dose.fits.gz
e00784-nd/e00784-nd-b00-f3000-t00000-g_dose.fits.gz
Key 'e00798-nd/e00798-nd-b00-f3000-t00000-g_dose.fits.gz

e01797-nd/e01797-nd-b00-f3000-t00000-g_dose.fits.gz
e01800-nd/e01800-nd-b00-f3000-t00000-g_dose.fits.gz
e01806-nd/e01806-nd-b00-f3000-t00000-g_dose.fits.gz
e01809-nd/e01809-nd-b00-f3000-t00000-g_dose.fits.gz
e01812-nd/e01812-nd-b00-f3000-t00000-g_dose.fits.gz
e01815-nd/e01815-nd-b00-f3000-t00000-g_dose.fits.gz
e01818-nd/e01818-nd-b00-f3000-t00000-g_dose.fits.gz
e01821-nd/e01821-nd-b00-f3000-t00000-g_dose.fits.gz
e01824-nd/e01824-nd-b00-f3000-t00000-g_dose.fits.gz
e01827-nd/e01827-nd-b00-f3000-t00000-g_dose.fits.gz
e01880-nd/e01880-nd-b00-f3000-t00000-g_dose.fits.gz
e01883-nd/e01883-nd-b00-f3000-t00000-g_dose.fits.gz
e01886-nd/e01886-nd-b00-f3000-t00000-g_dose.fits.gz
e01889-nd/e01889-nd-b00-f3000-t00000-g_dose.fits.gz
e01896-nd/e01896-nd-b00-f3000-t00000-g_dose.fits.gz
Key 'e01903-nd/e01903-nd-b00-f3000-t00000-g_dose.fits.gz' does not exist. Skipping...
e01903-nd/e01903-nd-b00-f3000-t00000-g_dose.fits.gz
Summed image saved as '01903_g100_image.fits'
     eclipse
129     1906
Key 'e0190

KeyError: 'Data must be a numpy array.'

In [ ]:
subset_list = sorted_folders[:1000]

groups_of_100 = [subset_list[i:i+100] for i in range(0, len(subset_list), 100)]


groups_of_100

In [28]:
full_groups

,eclipse
0,580
1,584
2,587
3,754
4,760
...,...
11458,46820
11459,46823
11460,46826
11461,46829


In [24]:
full_groups_33 = [full_groups[i:i+33] for i in range(0, 33, 3)]


In [25]:
full_groups_33

[    eclipse
 0       580
 1       584
 2       587
 3       754
 4       760
 5       767
 6       775
 7       784
 8       798
 9       807
 10      814
 11      826
 12      834
 13      841
 14      848
 15      861
 16      869
 17      882
 18      888
 19      899
 20      907
 21      920
 22      923
 23      926
 24      929
 25      932
 26      935
 27      938
 28      941
 29      944
 30      947
 31      953
 32      956,
     eclipse
 3       754
 4       760
 5       767
 6       775
 7       784
 8       798
 9       807
 10      814
 11      826
 12      834
 13      841
 14      848
 15      861
 16      869
 17      882
 18      888
 19      899
 20      907
 21      920
 22      923
 23      926
 24      929
 25      932
 26      935
 27      938
 28      941
 29      944
 30      947
 31      953
 32      956
 33      968
 34      971
 35      974,
     eclipse
 6       775
 7       784
 8       798
 9       807
 10      814
 11      826
 12      834
 13      8

In [29]:
full_groups = full_groups[0:130]
split_size = len(full_groups) // 3

# Split the DataFrame into smaller DataFrames
split_dfs = [full_groups.iloc[i:i+split_size] for i in range(0, len(full_groups), split_size)]

In [30]:
split_dfs

[    eclipse
 0       580
 1       584
 2       587
 3       754
 4       760
 5       767
 6       775
 7       784
 8       798
 9       807
 10      814
 11      826
 12      834
 13      841
 14      848
 15      861
 16      869
 17      882
 18      888
 19      899
 20      907
 21      920
 22      923
 23      926
 24      929
 25      932
 26      935
 27      938
 28      941
 29      944
 30      947
 31      953
 32      956
 33      968
 34      971
 35      974
 36      977
 37      980
 38      984
 39     1412
 40     1417
 41     1421
 42     1424,
     eclipse
 43     1427
 44     1430
 45     1433
 46     1436
 47     1439
 48     1442
 49     1447
 50     1450
 51     1453
 52     1456
 53     1490
 54     1505
 55     1517
 56     1520
 57     1529
 58     1532
 59     1535
 60     1542
 61     1566
 62     1573
 63     1580
 64     1587
 65     1594
 66     1606
 67     1614
 68     1617
 69     1620
 70     1623
 71     1626
 72     1633
 73     1640
 74     164